---

## Matchstick Experiment Hackerrank

---


In [46]:
tot_matchsticks = 2 * n * m - n - m
tot_ev = 0

for n_removed in range(tot_matchsticks + 1):
    # <s Compute probability of n_removed occurring
    n_combs = math.comb(n_removed, tot_matchsticks)
    prob_n_removed = n_combs * (p ** n_removed) * ((1 - p) ** (tot_matchsticks - n_removed))
    # /s>
    # <s Compute all possible scores
    all_scores = []
    for removed_combo in combinations(range(tot_matchsticks), n_removed):
        # <ss Score calculation
        score = compute_score(n, m, removed_combo)
        all_scores.append(score)
        # /ss>
    # <s Compute E.V. of all scores for this `n_removed`
    cur_ev = sum(all_scores) / len(all_scores) * prob_n_removed
    tot_ev += cur_ev

In [51]:
list(combinations(range(tot_matchsticks), n_removed))

[(0, 1, 2, 3)]

In [ ]:
# Define directions for checking adjacent cells (up, down, left, right)
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]


def compute_score(n, m, removed_combo):
    """Compute the score of the current game"""
    # Create an n x m grid with all boundaries intact (1 means boundary present)
    grid = [[1 for _ in range(2 * m + 1)] for _ in range(2 * n + 1)]

    # Remove matchsticks as specified by removed_combo
    for matchstick in removed_combo:
        # Translate matchstick index to grid position (example logic)
        grid_position = matchstick_to_grid_position(matchstick, n, m)
        grid[grid_position[0]][grid_position[1]] = 0  # Remove boundary

    # Find all connected components
    visited = set()
    connected_components_size_le_3 = 0

    for i in range(1, 2 * n, 2):  # Iterate over cell centers in the grid
        for j in range(1, 2 * m, 2):
            if (i, j) not in visited and grid[i][j] == 0:
                component_size = dfs(grid, i, j, visited)
                if component_size <= 3:
                    connected_components_size_le_3 += 1

    # Calculate the score
    score = connected_components_size_le_3 / (n * m)
    return score


# Helper function to map a matchstick index to grid position (implementation needed)
def matchstick_to_grid_position(matchstick_index, n, m):
    """
    Map a matchstick index to its position in the grid.
    Matchstick indices start from 0 and go up to (total matchsticks - 1).
    """
    total_horizontal = (n + 1) * m  # Total number of horizontal matchsticks

    if matchstick_index < total_horizontal:
        # It's a horizontal matchstick
        row = matchstick_index // m * 2  # Row index in the grid (even rows)
        col = (matchstick_index % m) * 2 + 1  # Column index in the grid (between cells)
        return (row, col)
    else:
        # It's a vertical matchstick
        vertical_index = matchstick_index - total_horizontal
        col = vertical_index // n * 2  # Column index in the grid (even columns)
        row = (vertical_index % n) * 2 + 1  # Row index in the grid (between cells)
        return (row, col)


def dfs(grid, x, y, visited):
    """Perform DFS to find the size of the connected component."""
    stack = [(x, y)]
    component_size = 0
    while stack:
        cx, cy = stack.pop()
        if (cx, cy) not in visited:
            visited.add((cx, cy))
            component_size += 1
            # Check adjacent cells
            for dx, dy in DIRECTIONS:
                nx, ny = cx + dx, cy + dy
                if (
                    is_valid(nx, ny, len(grid), len(grid[0]))
                    and (nx, ny) not in visited
                    and grid[nx][ny] == 0
                ):
                    stack.append((nx, ny))
    return component_size


def is_valid(x, y, n, m):
    """Check if (x, y) is within the grid boundaries."""
    return 0 <= x < n and 0 <= y < m

In [ ]:
# Example call to compute score
n = 2
m = 2
removed_combo = [(0, 1), (1, 2)]  # Example indices of removed matchsticks
score = compute_score(n, m, removed_combo)
print(f"Score: {score}")

---

## Multithreading

---

In [ ]:
import threading


def parallel_mean_std(lst, num_threads):
    n = len(lst)
    partial_sums = [0.0] * num_threads
    partial_sums_sq = [0.0] * num_threads
    threads = []

    # Worker function for each thread
    def worker(tid, start, end):
        sum_ = 0.0
        sum_sq = 0.0
        for i in range(start, end):
            x = lst[i]
            sum_ += x
            sum_sq += x * x
        partial_sums[tid] = sum_
        partial_sums_sq[tid] = sum_sq

    # Determine chunk size for each thread
    chunk_size = (n + num_threads - 1) // num_threads  # Ceiling division

    # Create and start threads
    for i in range(num_threads):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, n)
        t = threading.Thread(target=worker, args=(i, start, end))
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    # Combine partial results
    total_sum = sum(partial_sums)
    total_sum_sq = sum(partial_sums_sq)
    mean = total_sum / n
    variance = (total_sum_sq / n) - (mean * mean)
    std_dev = variance**0.5
    return mean, std_dev


# Example usage
if __name__ == "__main__":
    a = np.arange(0, 1e6, 1)
    mean, std_dev = parallel_mean_std(a, 8)
    print(f"Mean: {mean}")  # Output: Mean: 3.0
    print(f"Std Dev: {std_dev}")  # Output: Std Dev: 1.4142135623730951

---

## Gather, Reduce, Scatter

---

In [ ]:
def scatter(data, num_workers):
    chunk_size = (len(data) + num_workers - 1) // num_workers  # Ceiling division
    scattered_data = []
    for i in range(num_workers):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(data))
        data_chunk = data[start:end]
        scattered_data.append(data_chunk)
    return scattered_data

In [ ]:
def gather(worker_data):
    gathered_data = []
    for data_chunk in worker_data:
        gathered_data.extend(data_chunk)
    return gathered_data

In [ ]:
def allgather(worker_data):
    gathered_data = gather(worker_data)
    allgathered_data = [gathered_data.copy() for _ in worker_data]
    return allgathered_data

In [ ]:
def reduce(worker_data, operation="sum"):
    op_func = {
        "sum": lambda x, y: x + y,
        "max": lambda x, y: max(x, y),
        "min": lambda x, y: min(x, y),
    }.get(operation)

    if op_func is None:
        raise ValueError(f"Unsupported operation '{operation}'")

    reduced_result = None
    for data_chunk in worker_data:
        local_value = sum(data_chunk)  # Assuming summing up the chunk
        if reduced_result is None:
            reduced_result = local_value
        else:
            reduced_result = op_func(reduced_result, local_value)
    return reduced_result

In [ ]:
def allreduce(worker_data, operation="sum"):
    result = reduce(worker_data, operation)
    allreduced_data = [result for _ in worker_data]
    return allreduced_data

In [ ]:
"""Run it!"""

import random  # noqa: E402


num_workers = 4
total_elements = 1_000_000

# Generate a large dataset of 1,000,000 random numbers
data = [random.uniform(1, 100) for _ in range(total_elements)]
print(f"Total data elements: {len(data)}")

# 1. Scatter the data to worker nodes
worker_data = scatter(data, num_workers)
print(f"Data scattered to {num_workers} worker nodes.")

# 2. Each worker computes the square of its data elements
processed_worker_data = []
for idx, data_chunk in enumerate(worker_data):
    processed_chunk = [x**2 for x in data_chunk]
    processed_worker_data.append(processed_chunk)
    print(f"Worker {idx} processed its data chunk.")

# 3. Gather the processed data
gathered_data = gather(processed_worker_data)
print("Processed data gathered from all worker nodes.")

# 4. Perform reduction to compute the total sum of squares
total_sum_of_squares = reduce(processed_worker_data, operation="sum")
print(f"Total sum of squares computed using reduce: {total_sum_of_squares}")

# 5. Compute the average of the squares
average_of_squares = total_sum_of_squares / total_elements
print(f"Average of squares: {average_of_squares}")

# 6. Use allreduce to make the total sum available to all workers
allreduced_sums = allreduce(processed_worker_data, operation="sum")
print("Total sum of squares made available to all worker nodes using allreduce.")

# Each worker can compute the average using the allreduced sums
averages_at_workers = [total_sum / total_elements for total_sum in allreduced_sums]
for idx, avg in enumerate(averages_at_workers):
    print(f"Worker {idx} computed average of squares: {avg}")

# 7. Use allgather to share processed data among all worker nodes
allgathered_data = allgather(processed_worker_data)
for idx, data_at_worker in enumerate(allgathered_data):
    print(
        f"Worker {idx} has access to all processed data. Total elements: {len(data_at_worker)}"
    )

## CodeSignal

### Fitness Tracking System

Initially, you have a simplified fitness tracking system with the functionalities listed below. We are enhancing our system with additional features to schedule future fitness events and retrieve an agenda of these events.

- `add_activity(self, user_id: str, activity_type: str, distance: int) -> bool` - Adds a fitness activity for a user if it does not exist. Prevents duplicates by activity type.

- `update_activity(self, user_id: str, activity_type: str, distance: int) -> bool` - Updates the distance of an existing activity for a user.

- `get_activity(self, user_id: str, activity_type: str) -> int | None` - Retrieves the total distance of a specified activity for a user.

- `activity_summary(self, user_id: str) -> dict | None` - Provides a summary of all activities for a user, grouped by activity type.


New functionalities to be implemented:

- `schedule_event(self, timestamp: int, user_id: str, activity_type: str, distance: int, event_time: int) -> bool` - Schedules a future event for a user with a specified activity type and distance, ensuring it is for a future time. Returns True if the event is successfully scheduled and False otherwise. The timestamp parameter represents the current time.

- `get_agenda(self, user_id: str, from_time: int, to_time: int) -> list` - Retrieves a sorted list of scheduled fitness events for a user within a specified timeframe, from the starting to the ending time. The list should be sorted by the event_time field. Each event should be returned in the following form: {"activity_type": ..., "distance": …, "event_time": …}. If there are no events scheduled, returns empty list.

The `add_activity` method should be extended to integrate the distances from future scheduled events when adding an activity. Specifically, for `add_activity`, if there is a scheduled event for the same activity type, it should incorporate the distance from this future event into the distance parameter at the time of adding a new activity. This ensures that the system accounts for both immediate activities and those planned for the future right from the moment they are scheduled.

In [ ]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict

@dataclass
class FitnessTrackingSystem:
    users: dict = field(default_factory=dict)
    schedule: defaultdict = field(
        default_factory=lambda: defaultdict(lambda: SortedList(key=lambda x: x["event_time"]))
    )  # {user: [{"activity_type": ..., "distance": …, "event_time": …}]} (sorted by 'event_time')
    
    def add_activity(self, user_id: str, activity_type: str, distance: int) -> bool:
        if activity_type in self.users[user_id] or distance < 0:
            return False
        self.users[user_id][activity_type] = distance
        user_agenda = self.agenda[user_id]
        for activity in user_agenda:
            if activity["activity_type"] == activity_type:
                self.update_activity(user_id, activity["activity_type"], activity["distance"])
        return True

    def update_activity(self, user_id: str, activity_type: str, distance: int) -> bool:
        if user_id in self.users and activity_type in self.users[user_id]:
            self.users[user_id][activity_type] += distance
            return True
        return False

    def get_activity(self, user_id: str, activity_type: str) -> int:
        if user_id in self.users and activity_type in self.users[user_id]:
            return self.users[user_id][activity_type]
        return None

    def activity_summary(self, user_id: str) -> dict:
        if user_id in self.users:
            return self.users[user_id]
        return None
    
    def schedule_event(
        self, timestamp: int, user_id: str, activity_type: str, distance: int, event_time: int
    ) -> bool:
        if event_time <= timestamp:
            return False
        self.schedule[user_id].add(
            {"activity_type": activity_type, "distance": distance, "event_time": event_time}
        )
        return True
    
    def get_agenda(self, user_id: str, from_time: int, to_time: int) -> list:
        schedule = self.schedule[user_id]
        dummy_pre_event = {"event_time": from_time}
        dummy_post_event = {"event_time": to_time}
        l_i, r_i = schedule.bisect_left(dummy_pre_event), schedule.bisect_right(dummy_post_event)
        return schedule[l_i:r_i]
        

### Library Management System

You are given a library management system with the following functionality:

- `add_book(self, book_id: str, title: str) -> bool` — Adds a new book with the specified book_id and title. Returns True if the book is successfully added, False if the book already exists.

- `check_availability(self, book_id: str) -> str | None` — Checks if a book is available for borrowing. Returns the title if it is available, None if it is not.

- `borrow_book(self, user_id: str, book_id: str) -> bool` — Allows a user to borrow a book if it is available. The book is then marked as borrowed. Returns True if the process is successful, False if the book is not available or does not exist.

- `return_book(self, book_id: str) -> bool` — Returns a book to the library, making it available again. Returns True if the process is successful, False if the book does not exist.

- `get_borrow_history(self, book_id: str) -> list[str]` — Returns a list of user_ids who have borrowed the book in the order in which they borrowed it. Returns an empty list if the book has never been borrowed or does not exist.

Your new task is to extend this system to support the reservation of books that are currently borrowed and to notify users when their reserved books become available:

- `reserve_book(self, user_id: str, book_id: str) -> bool` — Allows a user to reserve a book that is currently borrowed. Returns True if the reservation is successful, meaning the book is currently borrowed and the user has not already reserved it. Returns False if the book does not exist or if the user has already reserved the book. Reservation means that whenever this book is returned by its current owner, the next person in the reservation queue borrows it and steps out of the reservation queue.

- `notify_availability(self, book_id: str) -> str | None` — Checks if there are any reservations for a book once it becomes available. Returns the user_id of the next user in line to be notified about the book's availability. If there are no reservations, returns None.

- `check_reservation(self, book_id: str) -> str | None` — Returns the user_id of the first user who has reserved the book, indicating who is next in line to borrow it. If the book has not been reserved by anyone, returns None.

In [ ]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict

@dataclass
class LibraryManagementSystem:
    books: defaultdict = field(default_factory=lambda: defaultdict(str))  # {book_id: title}
    borrow_status: defaultdict = field(default_factory=lambda: defaultdict(bool))  # {book_id: bool}
    borrow_history: defaultdict = field(default_factory=lambda: defaultdict(list))  # {book_id: list}
    res_queue: defaultdict = field(default_factory=lambda: defaultdict(list))  # {book_id: list}

    def add_book(self, book_id: str, title: str) -> bool:
        if book_id not in self.books:
            self.books[book_id] = title
            self.borrow_status[book_id] = False
            return True
        return False

    def check_availability(self, book_id: str) -> str | None:
        if book_id in self.books and not self.borrow_status[book_id]:
            return self.books[book_id]
        return None

    def borrow_book(self, user_id: str, book_id: str) -> bool:
        if book_id in self.books and not self.borrow_status[book_id]:
            self.borrow_status[book_id] = True
            if book_id not in self.borrow_history:
                self.borrow_history[book_id] = []
            self.borrow_history[book_id].append(user_id)
            return True
        return False

    def return_book(self, book_id: str) -> bool:
        if book_id in self.books and self.borrow_status[book_id]:
            self.borrow_status[book_id] = False
            if book_id in self.res_queue:
                new_borrower = self.notify_availability(book_id)
                self.res_queue[book_id].pop(0)
                self.borrow_book(new_borrower, book_id)
            return True
        return False

    def get_borrow_history(self, book_id: str) -> list[str]:
        if book_id in self.borrow_history:
            return self.borrow_history[book_id]
        return []
    
    def reserve_book(self, user_id: str, book_id: str) -> bool:
        if (self.borrow_status[book_id]) and (user_id not in self.res_queue[book_id]):
            self.res_queue[book_id] += [user_id]
            return True
        return False
    
    def notify_availability(self, book_id: str) -> str | None:
        if book_id in self.res_queue:
            return self.res_queue[book_id][0]
        return None
    
    def check_reservation(self, book_id: str) -> str | None:
        if book_id in self.res_queue:
            return self.res_queue[book_id][0]
        return None


### Social Media System

You are being asked to enhance a social media interaction system. Initially, your system supports operations related to posts and reactions, including creating posts, deleting them, reacting to posts, and summarizing reactions.

Here's a brief overview of what you have in your starter setup:

- `create_post(self, post_id: str, content: str) -> bool`: Creates a new post with a unique ID and content, returning True if successful.
- `delete_post(self, post_id: str) -> bool`: Deletes an existing post by ID, returning True if successful.
- `react_to_post(self, user_id: str, post_id: str, reaction_type: str) -> bool`: Allows users to react to a post, ensuring each user can react only once per post.
- `get_reaction_summary(self, post_id: str) -> dict | None`: Summarizes reactions to a post, returning a dictionary of reaction types and their counts.

Building upon this, you are tasked with adding functionality to support private groups. These groups allow users to share posts that are visible only to group members. This addition includes creating groups, adding members, sharing posts to groups, and fetching posts shared within a group. On top of that, you can no longer leave a reaction to a group post unless you belong to the corresponding group.

New functionalities to support:

- `create_group(self, group_id: str, operator_id`: str) -> bool: Creates a new group with a specified unique ID, the admin of the group is operator_id. Returns False if the group with such group_id already exists.
- `add_member(self, group_id: str, user_id: str, operator_id: str) -> bool`: Adds a new member to a group, actioned by the group's operator. If the operator_id doesn't match the actual group admin, return False.
- `share_post_to_group(self, post_id: str, group_id: str, user_id: str) -> bool`: Shares an existing post to a group's feed, available only if the user is a member of the group.
- `get_group_posts(self, group_id: str) -> list[str] | None`: Retrieves posts shared within a group, ordered by the time they were shared.

In [5]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict


@dataclass
class SocialMedia:
    posts: defaultdict = field(default_factory=lambda: defaultdict(dict))  # {post_id: dict}
    user_reactions: defaultdict = field(default_factory=lambda: defaultdict(dict))  # {post_id: dict}
    groups: defaultdict = field(
        default_factory=lambda: defaultdict(
            lambda: {"operator_id": "", "members": [], "posts": []}
        )
    )  # {group_id: dict}

    def create_post(self, post_id: str, content: str) -> bool:
        if post_id in self.posts:
            return False
        self.posts[post_id] = {"content": content, "reactions": {}}
        return True

    def delete_post(self, post_id: str) -> bool:
        if post_id not in self.posts:
            return False
        del self.posts[post_id]
        return True

    def react_to_post(self, user_id: str, post_id: str, reaction_type: str) -> bool:
        if post_id not in self.posts or user_id in self.user_reactions[post_id]:
            return False
        reactions = self.posts[post_id]["reactions"]
        reactions[reaction_type] = reactions.get(reaction_type, 0) + 1
        self.user_reactions[post_id][user_id] = reaction_type
        return True

    def get_reaction_summary(self, post_id: str):
        if post_id not in self.posts:
            return None
        return self.posts[post_id]["reactions"].copy()

    def create_group(self, group_id: str, operator_id: str) -> bool:
        if group_id in self.groups:
            return False
        self.groups[group_id]["operator_id"] = operator_id
        self.groups[group_id]["members"] = [operator_id]
        return True

    def add_member(self, group_id: str, user_id: str, operator_id: str) -> bool:
        if operator_id != self.groups[group_id]["operator_id"]:
            return False
        self.groups[group_id]["members"] += [user_id]
        return True

    def share_post_to_group(self, post_id: str, group_id: str, user_id: str) -> bool:
        if user_id in self.groups[group_id]["members"]:
            self.groups[group_id]["posts"] += [post_id]
            return True
        return False

    def get_group_posts(self, group_id: str) -> list[str] | None:
        return self.groups[group_id]["posts"]

### Task Management System

In this task, you are tasked with enhancing an existing task-tracking system by introducing new features for managing task dependencies. The system works with tasks, each having unique attributes: task_id, title, description, status, and priority.

The existing system supports the following operations:

- `add_task(self, task_id: str, title: str, description: str, status: str, priority: int) -> bool`: Adds a new task to the system.

- `update_task(self, task_id: str, title: Optional[str], description: Optional[str], status: Optional[str], priority: Optional[int]) -> bool`: Updates an existing task's attributes.

- `remove_task(self, task_id: str) -> bool`: Removes a task from the system.

- `get_tasks_by_status(self, status: str) -> List[str]`: Returns a list of task IDs filtered by status.

- `get_tasks_by_priority(self, priority: int) -> List[str]`: Returns a list of task IDs filtered by priority.

The upgraded system should support additional operations. These operations include:

- `add_dependency(self, task_id: str, dependency_task_id: str) -> bool`: Adds a dependency for a task, `task`, on another task, `dependency_task_id`. This method ensures that circular dependencies are not allowed to prevent tasks from being indefinitely blocked by each other.

- `remove_dependency(self, task_id: str, dependency_task_id: str) -> bool`: Removes an existing dependency of a task on another task.

- `get_dependent_tasks(self, task_id: str) -> List[str]`: Retrieves a list of tasks that are dependent on a given task.

Dependencies are described as follows: a task's status can automatically change to 'Open' once all tasks it depends on have been completed ('Closed'). Note, however, that when a task is removed from the system, the statuses of tasks that depend on the removed task are not automatically updated based on the removal action. Instead, automatic status updates occur in response to the completion of dependency tasks. This may necessitate manual status adjustments for some tasks when their dependency tasks are removed instead of completed.

When remove_task is called on a task with dependencies, it not only removes that task and its attributes but also updates the dependencies and dependents data structures to remove any references to the deleted task. However, the status of tasks that were dependent on the removed task is not automatically adjusted and may require manual intervention.

In [7]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict
from typing import List, Optional


@dataclass
class TaskTrackingSystem:
    tasks: defaultdict = field(default_factory=lambda: defaultdict(bool))  # {task_id: bool}
    task_attributes: defaultdict = field(default_factory=lambda: defaultdict(dict))  # {task_id: dict}
    task_dependencies: defaultdict = field(default_factory=lambda: defaultdict(list))  # {task_id: list}
    task_dependents: defaultdict = field(default_factory=lambda: defaultdict(list))  # {task_id: list}

    def add_task(
        self, task_id: str, title: str, description: str, status: str, priority: int
    ) -> bool:
        if task_id in self.tasks:
            return False
        self.tasks[task_id] = True
        self.task_attributes[task_id] = {
            "title": title,
            "description": description,
            "status": status,
            "priority": priority,
        }
        return True

    def update_task(
        self,
        task_id: str,
        title: Optional[str] = None,
        description: Optional[str] = None,
        status: Optional[str] = None,
        priority: Optional[int] = None,
    ) -> bool:
        if task_id not in self.tasks:
            return False

        if title is not None:
            self.task_attributes[task_id]["title"] = title

        if description is not None:
            self.task_attributes[task_id]["description"] = description

        if status is not None:
            self.task_attributes[task_id]["status"] = status
            if status.lower() == "closed":
                for task_id_parent in self.task_dependencies:
                    if task_id in self.task_dependencies[task_id_parent]:
                        all_children_closed = True
                        for child in self.task_dependencies[task_id_parent]:
                            if (
                                self.task_attributes[child]["status"].lower()
                                != "closed"
                            ):
                                all_children_closed = False
                        if all_children_closed:
                            self.update_task(task_id_parent, status="Open")

        if priority is not None:
            self.task_attributes[task_id]["priority"] = priority

        return True

    def remove_task(self, task_id: str) -> bool:
        if task_id in self.tasks:
            del self.tasks[task_id]
            del self.task_attributes[task_id]
            if task_id in self.task_dependencies:
                del self.task_dependencies[task_id]
                for task_id_parent in self.task_dependencies[task_id]:
                    self.task_dependents[task_id_parent].remove(task_id)
            if task_id in self.task_dependents:
                del self.task_dependents[task_id]
                for task_id_child in self.task_dependents[task_id]:
                    self.task_dependencies[task_id_child].remove(task_id)

            return True

        return False

    def get_tasks_by_status(self, status: str) -> List[str]:
        return sorted(
            [
                task_id
                for task_id, attrs in self.task_attributes.items()
                if attrs["status"] == status
            ]
        )

    def get_tasks_by_priority(self, priority: int) -> List[str]:
        return sorted(
            [
                task_id
                for task_id, attrs in self.task_attributes.items()
                if attrs["priority"] == priority
            ]
        )

    def add_dependency(self, task_id: str, dependency_task_id: str) -> bool:
        if (
            (task_id not in self.tasks)
            or (dependency_task_id not in self.tasks)
            or (task_id in self.task_dependencies[dependency_task_id])
        ):
            return False
        self.task_dependencies[task_id] += [dependency_task_id]
        self.task_dependents[dependency_task_id] += [task_id]
        return True

    def remove_dependency(self, task_id: str, dependency_task_id: str) -> bool:
        if (task_id in self.tasks) and (
            dependency_task_id in self.task_dependencies[task_id]
        ):
            self.task_dependencies[task_id].remove(dependency_task_id)
            return True
        return False

    def get_dependent_tasks(self, task_id: str) -> List[str]:
        return self.task_dependents[task_id]

### Voting System

Enhance a voting system with additional functionalities.

The starter code contains a set of basic functionalities:

- `register_candidate(self, candidate_id: str) -> bool`: This method is used for adding new candidates to our system.

- `vote(self, timestamp: int, voter_id: str, candidate_id: str) -> bool`: This method is designed to facilitate users casting their votes. Each vote is given a timestamp.

- `get_votes(self, candidate_id: str) -> int | None`: This method retrieves the total number of votes for a given candidate.

- `top_n_candidates(self, n: int) -> list[str]`: We also want to add a leaderboard functionality to our system. This method returns the top 'n' candidates sorted by the number of votes.

Add new functionalities:

- `get_voting_history(self, voter_id: str) -> dict | None`: Provides a detailed voting history for a specified voter, returning a dictionary mapping each candidate to the number of times voted for by the voter in question. Returns None if the voter ID does not exist.

- `block_voter_registration(self, timestamp: int) -> bool`: Implements a mechanism to halt any new voter registrations past a specified timestamp, effectively freezing the voter list as of that moment.

- `change_vote(self, timestamp: int, voter_id: str, old_candidate_id: str, new_candidate_id: str) -> bool`: Enables voters to change their vote from one candidate to another, given the change is made within a 24-hour window from their last vote, ensuring both the old and new candidates are registered, and that the voter initially voted for the old candidate.


In [8]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict

@dataclass
class VotingSystem:
    candidates: dict = field(default_factory=dict)
    voters: defaultdict = field(
        default_factory=lambda: defaultdict(lambda: {"votes": [], "timestamps": []})
    )

    def register_candidate(self, candidate_id: str) -> bool:
        if candidate_id in self.candidates:
            return False  # Candidate is already registered
        self.candidates[candidate_id] = 0  # Initialize candidates with 0 votes
        return True

    def vote(self, timestamp: int, voter_id: str, candidate_id: str) -> bool:
        # Check if block_time is set and if the vote attempt is after the block timestamp
        if hasattr(self, 'block_time') and timestamp >= self.block_time:
            return False  # Vote attempt is blocked due to the registration freeze

        if candidate_id not in self.candidates:
            return False  # Returns False if the candidate is not registered

        self.voters[voter_id]['votes'].append(candidate_id)  # Record the vote
        self.voters[voter_id]['timestamps'].append(timestamp)  # Record the time of the vote
        self.candidates[candidate_id] += 1  # Increment vote count for the candidate

        return True

    def get_votes(self, candidate_id: str) -> int | None:
        return self.candidates.get(
            candidate_id
        )  # Retrieve vote count for a candidate or None if not found

    def top_n_candidates(self, n: int) -> list[str]:
        return sorted(self.candidates, key=self.candidates.get)[-n:][::-1]

    def get_voting_history(self, voter_id: str) -> dict | None:
        if voter_id not in self.voters:
            return None
        voting_history = {candidate_id: self.voters[voter_id]['votes'].count(candidate_id) for candidate_id in set(self.voters[voter_id]['votes'])}
        return voting_history  # Returns a dictionary with each candidate voted for and the number of times they were voted for

    def block_voter_registration(self, timestamp: int) -> bool:
        self.block_time = timestamp  # Records the timestamp after which no registration is allowed
        return True
    
    def change_vote(self, timestamp: int, voter_id: str, old_candidate_id: str, new_candidate_id: str) -> bool:
        # Verify existence in the voting system
        if any(cid not in self.candidates for cid in [old_candidate_id, new_candidate_id]):
            return False

        # Check if the voter_id is valid and has previously voted
        if voter_id not in self.voters or not self.voters[voter_id]['votes']:
            return False  # Voter is either not registered or has not voted yet

        # Confirm voter has voted for the old candidate
        if old_candidate_id not in self.voters[voter_id]['votes']:
            return False

        # Ensure the operation is within the permitted timeframe
        last_vote_index = len(self.voters[voter_id]['votes']) - 1 - self.voters[voter_id]['votes'][::-1].index(old_candidate_id)
        if timestamp - self.voters[voter_id]['timestamps'][last_vote_index] > 86400:  # 24 hours in seconds
            return False

        # Perform the vote change
        # Remove the old vote
        del self.voters[voter_id]['votes'][last_vote_index]
        del self.voters[voter_id]['timestamps'][last_vote_index]
        
        # Append the new vote
        self.voters[voter_id]['votes'].append(new_candidate_id)
        self.voters[voter_id]['timestamps'].append(timestamp)

        # Update candidates' vote counts
        self.candidates[old_candidate_id] -= 1
        self.candidates[new_candidate_id] += 1

        return True

### Task Management System with Undo Capability

In this task, you will develop a sophisticated task management system with a focus on managing the lifecycle of tasks, including their creation, modification, and deletion. Special emphasis will be on extending the system with an undo feature and facilitating category changes for tasks.

The initial version of the task management system supports:

- `add_task(self, task_id: str, description: str) -> bool`: Adds a new task with a unique task_id and description. Returns True if the task is added successfully and False if task_id is already in use.

- `edit_task(self, task_id: str, new_description: str) -> bool`: Edits an existing task's description to new_description. Returns True if successful, False if task_id is not found.

- `remove_task(self, task_id: str) -> bool`: Removes the task with the specified task_id. Returns True if successful, False if task_id is not found.

- `set_task_priority(self, task_id: str, priority: int) -> bool`: Sets the priority of a task. Returns True if successful, False if task_id is not found.

- `categorize_task(self, task_id: str, category: str) -> bool`: Categorizes a task under a specified category. Returns True if successful, False if task_id is not found.

- `get_tasks_by_priority(self, min_priority: int) -> list`: Returns a list of task IDs that have a priority greater than or equal to min_priority.

- `get_tasks_by_category(self, category: str) -> list`: Returns a list of task IDs that belong to a specified category.

You need to add the following functionalities:

- `change_task_category(self, task_id: str, new_category: str) -> bool`: Changes the category of an existing task to `new_category`. Returns True if the category was changed successfully, and False if task_id is not found.

- `undo_last_operation(self) -> bool`: Undoes the most recent operation performed on the tasks. Returns True if an operation was undone, False otherwise.

The `undo_last_operation` should not undo a previous undo.

In [8]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict


@dataclass
class TaskManagementSystem:
    tasks: defaultdict = field(default_factory=lambda: defaultdict(dict))  # {task_id: dict}
    history: list = field(default_factory=list)  # list of tuple of dicts: [({"op": op}, {"arg1": arg1...})]

    def add_task(self, task_id: str, description: str, undo: bool = False) -> bool:
        if task_id in self.tasks:
            return False
        if not undo:
            h = ({"op": "add_task"}, {"task_id": task_id})
            self.history += [h]
        self.tasks[task_id] = {
            "description": description,
            "priority": None,
            "category": None,
        }
        return True

    def edit_task(self, task_id: str, new_description: str, undo: bool = False) -> bool:
        if task_id not in self.tasks:
            return False
        if not undo:
            h = (
                {"op": "edit_task"},
                {"task_id": task_id, "description": self.tasks[task_id]["description"]},
            )
            self.history += [h]
        self.tasks[task_id]["description"] = new_description
        return True

    def remove_task(self, task_id: str, undo: bool = False) -> bool:
        if task_id not in self.tasks:
            return False
        if not undo:
            h = (
                {"op": "remove_task"},
                {
                    "task_id": task_id,
                    "description": self.tasks[task_id]["description"],
                    "priority": self.tasks[task_id]["priority"],
                    "category": self.tasks[task_id]["category"],
                },
            )
            self.history += [h]
        del self.tasks[task_id]
        return True

    def set_task_priority(
        self, task_id: str, priority: int, undo: bool = False
    ) -> bool:
        if task_id not in self.tasks:
            return False
        if not undo:
            h = (
                {"op": "set_task_priority"},
                {"task_id": task_id, "priority": self.tasks[task_id]["priority"]},
            )
            self.history += [h]
        self.tasks[task_id]["priority"] = priority
        return True

    def get_tasks_by_priority(self, min_priority: int) -> list:
        return [
            task_id
            for task_id, task_info in self.tasks.items()
            if task_info["priority"] is not None
            and task_info["priority"] >= min_priority
        ]

    def categorize_task(self, task_id: str, category: str, undo: bool = False) -> bool:
        if task_id not in self.tasks:
            return False
        if not undo:
            h = (
                {"op": "categorize_task"},
                {"task_id": task_id, "category": self.tasks[task_id]["category"]},
            )
            self.history += [h]
        self.tasks[task_id]["category"] = category
        return True

    def get_tasks_by_category(self, category: str) -> list:
        return [
            task_id
            for task_id, task_info in self.tasks.items()
            if task_info["category"] == category
        ]

    def undo_last_operation(self) -> bool:
        if self.history:
            h = self.history[-1]
            match h[0]["op"]:
                case "add_task":
                    task_id = h[1]["task_id"]
                    self.remove_task(task_id, undo=True)
                case "edit_task":
                    task_id, description = h[1]["task_id"], h[1]["description"]
                    self.edit_task(task_id, description, undo=True)
                case "remove_task":
                    task_id, description = h[1]["task_id"], h[1]["description"]
                    priority, category = h[1]["priority"], h[1]["category"]
                    self.add_task(task_id, description, undo=True)
                    self.tasks[task_id]["priority"] = priority
                    self.tasks[task_id]["category"] = category
                case "set_task_priority":
                    task_id, priority = h[1]["task_id"], h[1]["priority"]
                    self.set_task_priority(task_id, priority, undo=True)
                case "categorize_task":
                    task_id, category = h[1]["task_id"], h[1]["category"]
                    self.categorize_task(task_id, category, undo=True)

            self.history.pop(-1)
            return True

        return False

    def change_task_category(
        self, task_id: str, new_category: str, undo: bool = False
    ) -> bool:
        if task_id not in self.tasks:
            return False
        if not undo:
            h = (
                {"op": "categorize_task"},
                {"task_id": task_id, "category": self.tasks[task_id]["category"]},
            )
            self.history += [h]
        self.tasks[task_id]["category"] = new_category
        return True

### Email System with Undo Capability

You are given a simplified email system that already supports basic functionalities such as sending emails, querying inboxes, and flagging emails. The following methods are already implemented:

- `send_email(self, from_user: str, to_user: str, subject: str, body: str) -> bool`: — Sends an email from from_user to to_user with the specified subject and body. This method always returns True in this implementation.

- `query_inbox(self, user: str) -> list[dict]`: — Returns a list of emails where user is the recipient, each email represented as a dictionary containing from, subject, and body.

- `sent_emails_count(self) -> dict`: — Returns a dictionary listing the number of emails sent by each user.

- `flag_email(self, user: str, subject: str) -> bool`: — Marks an email with the given subject in user's inbox as flagged. It returns True if successful and False otherwise.

Now, your task is to enhance this system by introducing the capability to undo sent emails and to log users out:

- `undo_send(self, from_user: str, subject: str) -> bool`: — Reverses the last email sent by from_user with the specific subject, removing it from the recipient's inbox. Only the most recent email with that subject is affected. It returns True if successful and False otherwise.

- `logout_user(self, user: str) -> bool`: — Deactivates user's account, preventing any further emails from being sent from or received by this user until reactivated. It returns True if successful and False otherwise.

Sending an email to or from a logged-out user should now result in `send_email` returning False

In [9]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict


@dataclass
class EmailSystem:
    inboxes: defaultdict = field(default_factory=lambda: defaultdict(lambda: []))  # {str: list(dict)}
    sent_counts: defaultdict = field(default_factory=lambda: defaultdict(lambda: 0))  # {str: int}
    sent_emails: defaultdict = field(default_factory=lambda: defaultdict(lambda: []))  # {str: list(dict)}
    logged_out: set = field(default_factory=set)

    def send_email(self, from_user: str, to_user: str, subject: str, body: str) -> bool:
        if to_user in self.logged_out or from_user in self.logged_out:
            return False
        if to_user not in self.inboxes:
            self.inboxes[to_user] = []
        # Add the email to the recipient's inbox
        self.inboxes[to_user].append({"from": from_user, "subject": subject, "body": body})
        # Increment sent email count
        self.sent_counts[from_user] = self.sent_counts.get(from_user, 0) + 1
        # Record the email in sent_emails for undo functionality
        if from_user not in self.sent_emails:
            self.sent_emails[from_user] = []
        self.sent_emails[from_user].append({"to_user": to_user, "subject": subject, "body": body})
        return True

    def query_inbox(self, user: str) -> list:
        return self.inboxes.get(user, [])

    def sent_emails_count(self) -> dict:
        return self.sent_counts

    def flag_email(self, user: str, subject: str) -> bool:
        if user in self.inboxes:
            for email in self.inboxes[user]:
                if email["subject"] == subject:
                    email["flagged"] = True
                    return True
        return False

    def undo_send(self, from_user: str, subject: str) -> bool:
        for i, from_email in enumerate(self.sent_emails[from_user]):
            if subject in from_email["subject"]:
                # Remove for `from_user`
                self.sent_emails[from_user].pop(i)
                self.sent_counts[from_user] -= 1
                if self.sent_counts[from_user] == 0:
                    self.sent_counts.pop(from_user)
                # Remove for `to_user`
                to_user = from_email["to_user"]
                for j, to_email in enumerate(self.inboxes[to_user]):
                    if subject in to_email["subject"]:
                        self.inboxes[to_user].pop(j)

                return True

        return False

    def logout_user(self, user: str) -> bool:
        self.logged_out.add(user)
        return True

### Note-taking application with undo capability

In this task, your objective is to develop a note-taking application from the ground up, incorporating key functionalities such as creating and retrieving notes, tagging notes, sharing them among users with varying access levels, as well as introducing versioning for notes and an undo functionality to revert notes to their previous versions.

The application should support the following functionalities:

- `add_note(self, title: str, content: str) -> str` — Creates a new note with the specified title and content, and returns a unique identifier for the note.

- `get_note(self, note_id: str) -> dict` — Retrieves a note by its unique identifier. Returns a dictionary containing the title and content of the note.

- `add_tag(self, note_id: str, tag: str) -> bool` — Associates a tag with a specific note. Returns True if the operation is successful and False otherwise.

- `get_notes_by_tag(self, tag: str) -> list` — Returns a list of all notes associated with a specified tag. Each note is represented as a dictionary containing its identifier, title, and content.
share_note(self, note_id: str, user_id: str, access_level: str) -> bool — Shares a note with another user by specifying a unique identifier, the user's identifier, and the access level (read or write). Returns True if the operation is successful and False otherwise.

- `get_shared_notes(self, user_id: str) -> list` — Retrieves all notes shared with a specific user. Returns a list of dictionaries, each containing the note's identifier, title, content, and the user's access level.

New Features to Implement:

- `add_version(self, note_id: str, title: str, content: str, tags: list) -> bool` — This method introduces a new version of the note by updating its title and/or content. If the operation is successful, all previous versions of the note should be preserved, and the method will return True. It returns False if either the note does not exist or no modifications were detected.

- `undo_version(self, note_id: str) -> bool` — This method reverts a note to its immediate previous version. It returns True if the undo operation is successful. If the note is already in its initial version or does not exist, it returns False.

In [ ]:

from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict

@dataclass
class NoteTakingApp:
    notes: defaultdict = field(default_factory=lambda: defaultdict(lambda: defaultdict(list)))  # {note_id: dict}
    tags: defaultdict = field(default_factory=lambda: defaultdict(list))  # {tag: list}
    shares: defaultdict = field(default_factory=lambda: defaultdict(lambda: defaultdict(str)))
    versions: defaultdict = field(default_factory=lambda: defaultdict(list))  # {note_id: [notes]}  e.g. versions["1"][{...}, {...}] -> []
    version_idx: defaultdict = field(default_factory=lambda: defaultdict(int))

    def add_note(self, title: str, content: str) -> str:
        note_id = str(len(self.notes) + 1)
        self.notes[note_id] = {"title": title, "content": content, "tags": []}
        self.versions[note_id] += [self.notes[note_id].copy()]
        return note_id
        

    def get_note(self, note_id: str) -> dict:
        if note_id in self.notes:
            return {"title": self.notes[note_id]["title"], "content": self.notes[note_id]["content"]}
        return None

    def add_tag(self, note_id: str, tag: str) -> bool:
        if note_id in self.notes:
            self.notes[note_id]["tags"].append(tag)
            if tag in self.tags:
                self.tags[tag].append(note_id)
            else:
                self.tags[tag] = [note_id]
            return True
        return False

    def get_notes_by_tag(self, tag: str) -> list:
        if tag in self.tags:
            return [
                {
                    "note_id": note_id,
                    "title": self.notes[note_id]["title"], 
                    "content": self.notes[note_id]["content"]
                }
                for note_id in self.tags[tag]
            ]
        return []

    def share_note(self, note_id: str, user_id: str, access_level: str) -> bool:
        if note_id in self.notes and access_level in ["read", "write"]:
            if note_id not in self.shares:
                self.shares[note_id] = {}
            self.shares[note_id][user_id] = access_level
            return True
        return False

    def get_shared_notes(self, user_id: str) -> list:
        result = []
        for note_id, users in self.shares.items():
            if user_id in users:
                access_level = users[user_id]
                result += [
                    {
                        "note_id": note_id,
                        "title": self.notes[note_id]["title"],
                        "content": self.notes[note_id]["content"],
                        "access_level": access_level
                    }
                ]
        return result
    
    def add_version(self, note_id: str, title: str, content: str, tags: list) -> bool:
        if (
            self.notes[note_id]["title"] == title
            and self.notes[note_id]["content"] == content
            and self.notes[note_id]["tags"] == tags
        ):
            return False
        self.notes[note_id] = {"title": title, "content": content, "tags": tags}
        self.versions[note_id] += [self.notes[note_id].copy()]
        self.version_idx[note_id] += 1
        return True
        
    
    def undo_version(self, note_id: str) -> bool:
        if len(self.versions[note_id]) == 1:
            return False
        self.version_idx[note_id] -= 1
        self.notes[note_id] = self.versions[note_id][self.version_idx[note_id]]
        return True


### Social media platform with undo capability

In this task, you'll develop a social media platform from the ground up, which includes capabilities for handling user posts and comments, along with newer functionalities to edit comments and a system-wide rollback mechanism for edits made to posts and comments. All features need to be integrated while ensuring backward compatibility.

The following methods are already implemented:

create_post(self, post_id: str, content: str, timestamp: int) -> bool - Creates a new post with the given content and timestamp. Returns True if the post is successfully created and False if a post with the same ID already exists.
edit_post(self, post_id: str, new_content: str) -> bool - Edits the content of an existing post. Returns True if the edit is successful and False if the post does not exist. This method doesn't change the timestamp of the post.
delete_post(self, post_id: str) -> bool - Deletes a post by its id. Returns True if the deletion is successful and False if the post does not exist.
like_post(self, post_id: str) -> bool - Increments the like count of a post. Returns True if successful and False if the post does not exist.
top_n_liked_posts(self, n: int) -> list - Returns a list of the n most liked posts, sorted by likes in descending order, breaking ties by timestamp.
comment_on_post(self, post_id: str, comment_id: str, content: str, timestamp: int) -> bool - Adds a comment to a post. Returns True if successful, False if either the post does not exist or a comment with the same id already exists.
top_n_commented_posts(self, n: int) -> list - Returns a list of the n most commented posts, sorted by the number of comments in descending order, breaking ties by timestamp.
Your task, however, is to implement the new functionalities:

edit_comment(self, post_id: str, comment_id: str, new_comment: str) -> bool - Allows users to modify their comments on posts. Each comment is uniquely identified within its post by a comment_id. Returns True if successful, False if the post or the comment does not exist. Editing comment doesn't change the post's timestamp.
rollback_post_edit(self, post_id: str) -> bool - Reverses the last edit made to a post's content, restoring its previous content. Returns True if there is an edit to rollback and operation is successful, otherwise False. This method changes only post's content, leaving likes, comments and everything else intact.
rollback_comment_edit(self, post_id: str, comment_id: str) -> bool - Reverses the last edit made to a comment's content, restoring its previous content. Returns True if there is an edit to rollback and operation is successful, otherwise False.

In [ ]:
class SocialMediaPlatform:
    def __init__(self):
        # Store posts and comments separately, as the tests
        # expect self.posts[...] to hold posts
        # and self.comments[...] to hold comments.
        self.posts = (
            {}
        )  # post_id -> { content, timestamp, likes, comments (list), edit_history (list) }
        self.comments = {}  # comment_id -> { content, timestamp, edit_history (list) }

    def create_post(self, post_id: str, content: str, timestamp: int) -> bool:
        """
        Creates a new post with the given content and timestamp.
        Returns True if the post is created successfully and
        False if a post with the same ID already exists.
        """
        if post_id in self.posts:
            return False
        self.posts[post_id] = {
            "content": content,
            "timestamp": timestamp,
            "likes": 0,
            "comments": [],  # store list of comment IDs
            "edit_history": [],  # stack of previous contents
        }
        return True

    def edit_post(self, post_id: str, new_content: str) -> bool:
        """
        Edits the content of an existing post, without changing its timestamp.
        Pushes the old content onto edit_history so it can be rolled back.
        Returns True if successful, False if the post does not exist.
        """
        if post_id not in self.posts:
            return False

        old_content = self.posts[post_id]["content"]
        self.posts[post_id]["edit_history"].append(old_content)
        self.posts[post_id]["content"] = new_content
        return True

    def rollback_post_edit(self, post_id: str) -> bool:
        """
        Reverses the last edit made to a post, restoring its previous content.
        Returns True if there's an edit to roll back, False otherwise.
        """
        if post_id not in self.posts:
            return False
        if len(self.posts[post_id]["edit_history"]) == 0:
            return False

        last_content = self.posts[post_id]["edit_history"].pop()
        self.posts[post_id]["content"] = last_content
        return True

    def delete_post(self, post_id: str) -> bool:
        """
        Deletes a post by its ID. Returns True if successful,
        False if the post does not exist.  (Not specifically tested.)
        """
        if post_id not in self.posts:
            return False

        # Optionally, could delete the comments belonging to it, but tests do not check this.
        # for cid in self.posts[post_id]['comments']:
        #     if cid in self.comments:
        #         del self.comments[cid]
        del self.posts[post_id]
        return True

    def like_post(self, post_id: str) -> bool:
        """
        Increments the like count of a post.
        Returns True if successful and False if the post does not exist.
        """
        if post_id not in self.posts:
            return False

        self.posts[post_id]["likes"] += 1
        return True

    def top_n_liked_posts(self, n: int) -> list:
        """
        Returns the IDs of the n most-liked posts, sorted by
        descending likes. Ties in 'likes' are broken by descending timestamp.
        """
        # Sort all post_ids by (-likes, -timestamp)
        sorted_posts = sorted(
            self.posts.keys(),
            key=lambda pid: (-self.posts[pid]["likes"], -self.posts[pid]["timestamp"]),
        )
        return sorted_posts[:n]

    def comment_on_post(
        self, post_id: str, comment_id: str, content: str, timestamp: int
    ) -> bool:
        """
        Adds a new comment (with ID = comment_id) to an existing post.
        Returns True if successful; False if the post doesn't exist
        or the comment_id is already in use.
        """
        if post_id not in self.posts:
            return False
        if comment_id in self.comments:
            return False

        # Create the comment in self.comments
        self.comments[comment_id] = {
            "content": content,
            "timestamp": timestamp,
            "edit_history": [],
        }
        # Attach the comment_id to the post's comment list
        self.posts[post_id]["comments"].append(comment_id)
        return True

    def edit_comment(self, post_id: str, comment_id: str, new_comment: str) -> bool:
        """
        Edits an existing comment, without changing its timestamp.
        Pushes the old comment content onto its edit_history so it can be rolled back.
        Returns True if successful, False if the post or comment does not exist.
        """
        if post_id not in self.posts:
            return False
        if comment_id not in self.posts[post_id]["comments"]:
            return False

        old_content = self.comments[comment_id]["content"]
        self.comments[comment_id]["edit_history"].append(old_content)
        self.comments[comment_id]["content"] = new_comment
        return True

    def rollback_comment_edit(self, post_id: str, comment_id: str) -> bool:
        """
        Reverses the last edit to a comment, restoring its previous content.
        Returns True if there's an edit to roll back; False otherwise.
        """
        if post_id not in self.posts:
            return False
        if comment_id not in self.posts[post_id]["comments"]:
            return False

        if len(self.comments[comment_id]["edit_history"]) == 0:
            return False

        last_comment_content = self.comments[comment_id]["edit_history"].pop()
        self.comments[comment_id]["content"] = last_comment_content
        return True

    def top_n_commented_posts(self, n: int) -> list:
        """
        Returns a list of the n most-commented post IDs,
        sorted by descending number of comments.
        Ties in 'number of comments' are broken by descending timestamp.
        """
        sorted_posts = sorted(
            self.posts.keys(),
            key=lambda pid: (
                -len(self.posts[pid]["comments"]),
                -self.posts[pid]["timestamp"],
            ),
        )
        return sorted_posts[:n]

### Ticket management system with merge and split functionalities

### Sorting emails

Imagine you have a large mailbox that receives emails from various sources and you need to organize these emails. Your task involves implementing a Python function named `organize_inbox()`. This function will accept a string of emails as input and output a list of tuples. Each tuple contains two elements: the sender's email address and the total count of emails received from this sender.

Each email is represented by various metadata separated by commas, such as `"sender_email_address, Subject, Timestamp"`. The total string comprises these entries, separated by semicolons. Emails originate from distinct senders and can occur at any timestamp in the "HH:MM" format within a 24-hour range.

Here is the format of the string: `"sender_email_dddress1, Subject1, 09:00; sender_email_address2, Subject2, 10:00; sender_email_dddress1, Subject3, 12:00"`

The function should return: `[("sender_email_dddress1", 2), ("sender_email_dddress2", 1)].`

Your function must extract the sender's email address and count the number of emails received from each sender, outputting a list of tuples. Each tuple should contain the sender's email address, followed by the count of emails received from them. The tuples should be sorted by the descending order of these counts. If two senders have sent the same number of emails, the tuples should be listed in ascending order based on the senders' email addresses.

The sender's email address is always followed by a comma, a space, and the start of the subject line. The subject line is always followed by a comma, a space, and the timestamp. All emails are unique, meaning there will be no emails with the same subject and timestamp from the same sender.

In [10]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict

def organize_inbox(inbox_string):
    emails = inbox_string.split(";")
    sent_ct = defaultdict(int)
    for email in emails:
        sender_address, _subject, _ts = email.split(",")
        sent_ct[f"{sender_address.strip()}"] += 1
    
    result = []
    for item in sent_ct.items():
        result.append(item)
    # Sort first on sent count (negative for descending order), then (in case of ties) on sender address
    return sorted(result, key=lambda x: (-x[1], x[0]))


### Sorting Programming Competition Logs

There is a school hosting an online programming competition. Each problem is assigned a unique level of difficulty. Every time a student successfully solves a problem, their score is updated based on the problem's difficulty level. However, if a student makes an unsuccessful attempt, they incur a penalty. The competition logs every action of each student in a string.

Your task is to create a Python function named `analyze_competition()`. It will take a string of logs as input and output a list of tuples, representing the students' score, the number of successful attempts, and the total penalties. The tuples should be sorted by the decreasing order of scores of their respective students. It is guaranteed that there will be no students with the same positive score. Don't include students in the output who haven't solved any problem.

For example, if you have logs like this:
`"1 solve 09:00 50, 2 solve 10:00 60, 1 fail 11:00, 3 solve 13:00 40, 2 fail 14:00, 3 solve 15:00 70"`,
your function should return: `[(3, 110, 2, 0), (2, 60, 1, 1), (1, 50, 1, 1)]`.

All log entries are separated by a comma and a space. It is guaranteed that the log entries are sorted in chronological order.

In [2]:
from dataclasses import dataclass, field
from collections import defaultdict, deque, Counter
from sortedcontainers import SortedList, SortedSet, SortedDict


def analyze_competition(logs: str) -> list[tuple]:  # return list of 4-tuple
    logs = logs.split(",")
    scores = defaultdict(lambda: {"score": 0, "solve": 0, "fail": 0})
    for log in logs:
        log = log.strip().split(" ")
        uid, _outcome, _ts = log[0], log[1], log[2]
        if len(log) > 3:  # solve
            score = log[3]
            scores[uid]["score"] += int(score)
            scores[uid]["solve"] += 1
        else:  # fail
            scores[uid]["fail"] += 1

    result = []
    for uid in list(scores.keys()):
        if scores[uid]["solve"] < 1:
            scores.pop(uid)
        else:
            result += [
                (
                    int(uid),
                    scores[uid]["score"],
                    scores[uid]["solve"],
                    scores[uid]["fail"],
                )
            ]
    return sorted(result, key=lambda x: -x[1])

### Sorting Books by Borrowed-time

You are provided with log data from a library's digital system, stored in string format. The log represents books' borrowing activities, including the book ID and the time a book is borrowed and returned. The structure of a log entry is as follows: `"<book_id> borrow <time>"` or `"<book_id> return <time>"`

The time is given in the HH:MM 24-hour format, and the book ID is a positive integer between 1 and 500. The logs are separated by a comma, followed by a space (", ").

Your task is to create a Python function named `solution()`. This function will take as input a string of logs and output a list of tuples representing the books with the longest borrowed duration. Each tuple contains two items: the book ID and the book's borrowed duration. By 'borrowed duration,' we mean the period from when the book was borrowed until it was returned. If a book has been borrowed and returned multiple times, the borrowed duration is the total cumulative sum of those durations. If multiple books share the same longest borrowed duration, the function should return all such books in ascending order of their IDs.

For example, if we have a log string as follows: `"1 borrow 09:00, 2 borrow 10:00, 1 return 12:00, 3 borrow 13:00, 2 return 15:00, 3 return 16:00"`,

the function will return: `[(2, '05:00')]`.

Note: You can safely assume that all borrowing actions for a given book will have a corresponding return action in the log, and vice versa. Also, the logs are sorted by the time of the action.

In [4]:
from collections import defaultdict, deque, Counter
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from sortedcontainers import SortedList, SortedSet, SortedDict


def solution(logs: str):
    time_fmt = "%H:%M"
    logs = logs.split(",")
    borrow_time = defaultdict(timedelta)  # {book_id: duration}
    borrowed = {}  # {book_id: time}
    for log in logs:
        log = log.strip().split(" ")
        book_id, status, ts = log[0], log[1], log[2]
        if status.lower() == "borrow":
            borrowed[book_id] = ts
        elif status.lower() == "return":
            td = datetime.strptime(ts, time_fmt) - datetime.strptime(
                borrowed[book_id], time_fmt
            )
            borrow_time[book_id] += td
    max_borrow_time = max(borrow_time.values())
    result = []
    for item in borrow_time.items():
        if item[1] == max_borrow_time:
            hours, remainder = divmod(item[1].seconds, 3600)
            mins = remainder // 60
            fmt_td = f"{str(hours).zfill(2)}:{str(mins).zfill(2)}"
            result += [(int(item[0]), fmt_td)]

    return result

### Binary search in sorted array traversal

You are given two arrays, Array A and Array B, each containing `n` integers.

Your task is to create a function `optimizedReplace` that returns a new array, `C`. For each index `i`, `C[i]` should contain a specific value from `A` (`A[j]`), determined by the condition that `B[i]` is the closest number to `B[j]`.

This means that `C` will have the same length as `A` and `B`. `C[i]` will contain the value corresponding to the jth index from `A`, where the value at the jth index in `B` is closest to the value at the ith index of `B`.

Assume that there is no ambiguity: `B` is given in a way that no two elements have the same minimal absolute difference with `B[i]`.

Remember, the number of elements, `n`, and the range of the elements, imply that brute force solutions will not be efficient. You should aim to leverage optimized algorithms and techniques to solve this task efficiently.

Example

Suppose we have `A = [10, 20, 30, 40, 50]` `B = [7, 5, 1, 2, 4]`.

The function `optimizedReplace(A, B)` should work as follows:

For `B[0] = 7`, the closest number in `B` is 5 at index 1. Hence, `C[0] = A[1] = 20`.

For `B[1] = 5`, the closest number in `B` is 4 at index 4. Thus, `C[1] = A[4] = 50`.

...

Lastly, for `B[4] = 4`, the closest number in B is 5 at index 1. Therefore, `C[4] = A[1] = 20`.

Thus, `optimizedReplace([10, 20, 30, 40, 50], [7, 5, 1, 2, 4])` should return `[20, 50, 40, 30, 20]`.

In [1]:
from collections import defaultdict, deque, Counter
from dataclasses import dataclass, field
from sortedcontainers import SortedList, SortedSet, SortedDict

def optimizedReplace(A, B):
    if len(B) == 1:
        return A

    C = [0] * len(B)
    # Create a sorted list of (value, original_index) pairs
    B_sorted = SortedList((val, idx) for idx, val in enumerate(B))

    for i, num in enumerate(B):
        # Remove the current element so we don't pick ourselves as the closest
        B_sorted.remove((num, i))
        # Find the insertion position for (num, i) in B_sorted
        pos = B_sorted.bisect_left((num, i))
        # Gather up to two candidate idxs (left and right of num); only one if `num` is max or min of B
        candidates = []
        if pos > 0:  # if not, it's the min
            candidates.append(B_sorted[pos - 1])  # left idx
        if pos < len(B_sorted):  # if not, it's the max
            candidates.append(B_sorted[pos])  # right idx
        closest_val, closest_idx = min(candidates, key=lambda x: abs(x[0] - num))
        C[i] = A[closest_idx]
        
        # Re-add current element
        B_sorted.add((num, i))
    
    return C

### Min Max Block Length

Given an array, find the particular integer `k` in the array that minimizes the maximum block length of the blocks formed by splitting the array on the removals of `k`.

e.g. if `arr = [1, 2, 2, 3, 1, 4, 4, 4, 1, 2, 5]`, we should find `k=1` because splitting on `k=1`
yields blocks of `[2, 2, 3], [4, 4, 4], [2, 5]`, so the maximal block length is 3 (splitting on `k={2,3,4,5}` yields block lengths > 3).

In [18]:
from collections import defaultdict, deque, Counter
from dataclasses import dataclass, field
from sortedcontainers import SortedList, SortedSet, SortedDict

# For each unique element, get every position of the element, then track and save
# the max block size when splitting the array into blocks as we traverse the array.

# Sort the `max_block_sz` hashmap that contains {unique_element: block_size} by values, and return
# the min value and corresponding key.

def min_max_block(arr) -> tuple:  # return (ele, max_block_size)
    latest_idx = defaultdict(lambda: -1)
    max_block_sz = defaultdict(int)

    # Handle all blocks up to tail block for each unique element
    for i, ele in enumerate(arr):
        if max_block_sz[ele] < 0:  # first block for `ele`
            max_block_sz[ele] = i  
        elif (i - latest_idx[ele]) > max_block_sz[ele]:  # subsequent blocks for `ele`
            max_block_sz[ele] = i - latest_idx[ele] - 1
        latest_idx[ele] = i

    # Handle tail block for each unique element
    n = len(arr)
    for ele, last_idx in latest_idx.items():
        max_block_sz[ele] = max(max_block_sz[ele], (n- last_idx - 1))

    return sorted(max_block_sz.items(), key=lambda x: x[1])[0]

### Partitioning substrings

In the legendary kingdom of Alphaland, exists a cryptic scroll named`s`, inscribed with a string of elegant lowercase letters. As a renowned linguist, your quest is to decipher this scroll by partitioning it into as many chapters (substrings) as your wit allows, under the condition that each letter graces only a maximum of one chapter, with a function called `string_partition(s: str) -> list[int]`.

This is a sacred task, for the chapters must follow the progression of the original text, reflecting the flow of the narrative embedded within it. 

To illustrate, if you were to partition the text "abacdcd" into the chapters "aba" and "cdcd", your resultant report of chapter lengths should list as [3, 4], preserving the original sequence.

The text "abacdabcd" would return a single length of [9].

In [1]:
def string_partition(s):
    # First pass through `s`: find last occurrence idx of each char
    last_index = {c: i for i, c in enumerate(s)}
    # Second pass through `s`: check if the furthest last occurrence in the
    # current substring is equal to the current index, if so, split
    result = []
    last_split_i, max_last = 0, 0
    for i, ch in enumerate(s):
        # Update the furthest last occurrence
        max_last = max(max_last, last_index[ch])
        # When we reach the max_last, close the partition
        if i == max_last:
            result += [(i - last_split_i + 1)]
            last_split_i = i + 1

    return result

### Minimum distance between substrings in a string

In [2]:
from collections import defaultdict

def solution(word_list):
    word_idxs = defaultdict(list)
    word_distance = defaultdict(lambda: 1e6)
    
    for i, word in enumerate(word_list):
        word_idxs[word] += [i]
        if len(word_idxs[word]) > 1:
            dist = word_idxs[word][-1] - word_idxs[word][-2]
            word_distance[word] = min(word_distance[word], dist)
    
    word_distance = sorted(word_distance.items(), key=lambda x: x[1])
    return word_distance[0]

### Two-pointer sorted array traversal

Find all possible pairs of elements in a sorted array that sum up to a given target value.

In [2]:
def find_pairs(numbers, target):
    numbers.sort()
    left, right = 0, len(numbers) - 1  # two pointers
    pairs = []  # list of two-tuples

    while left < right:
        total = numbers[left] + numbers[right]

        if total == target:  # pair found; move both bounds closer together
            pairs += [([numbers[left], numbers[right]])]
            left += 1
            right -= 1
        elif total < target:  # move lower bound up
            left += 1
        elif total > target:  # move upper bound down
            right -= 1
    return pairs

### Find maximum subarray sum of size `k` in array of length `n`

Two pointer approach

In [1]:
def maximum_sum(numbers, k):
    left_ptr, right_ptr = 0, k
    cur_sum, max_sum = sum(numbers[0:k]), sum(numbers[0:k])
    max_sum_i = k - 1  # currently, index window on right bound
    for i in range(k, len(numbers), 1):
        # Take cur window, subtract ele moving out (left ptr), add ele moving in (right ptr)
        cur_sum = cur_sum - numbers[left_ptr] + numbers[right_ptr]
        max_sum, max_sum_i = (cur_sum, i) if cur_sum > max_sum else (max_sum, max_sum_i)
        left_ptr += 1
        right_ptr += 1

    return (max_sum, max_sum_i - k + 1)  # change window index to left bound

### Find longest contiguous subarray sum equal to `k` in an array

Two pointer approach

In [2]:
def get_longest_subarray(array, k):
    
    cur_target_idxs = [0, -1]
    reached = False
    left_ptr, right_ptr = 0, 0
    cur_sum = 0
    
    while right_ptr < len(array):
        
        cur_sum += array[right_ptr]
        
        while cur_sum > k and left_ptr < right_ptr:
            cur_sum -= array[left_ptr]
            left_ptr += 1
        
        if cur_sum == k:
            if (right_ptr - left_ptr) > (cur_target_idxs[1] - cur_target_idxs[0]):
                cur_target_idxs[1], cur_target_idxs[0] = right_ptr, left_ptr
                reached = True
        
        right_ptr += 1
    
    if reached:
        return array[cur_target_idxs[0] : (cur_target_idxs[1] + 1)]
    else:
        return []

### Building an array `C` from an array `A` based on values in array `B`

Suppose you have two equally long arrays, A and B, with a length varying from 1 to 1000, with each element being a unique positive integer ranging from 1 up to 10^6. Your challenge is to craft a Python function that identifies the closest number in array B to 2 * B[i] for each i. Once this number is identified, say for the specific i it is B[j], we want to create an array from A[j]s in the order of increasing is.

For example, if we have:

```python
A = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
B = [4, 12, 3, 9, 6, 1, 5, 8, 37, 25, 100]
```

then

```python
C = [80, 100, 50, 20, 20, 60, 40, 20, 110, 90, 110]
```

The first item in B is 4 at index=0. Double of this number is 8. The closest number to 8 in array B is 8 which is at index=7. The number at the same index in array A is 80, so we add 80 to our new array.

The second item in B is 12 at index=1. Double of this number is 24. The closest number to 24 in B is 25 which is at index=9. Corresponding index in A has the number 100. So, we add 100 to our new array.

...

The last item in B is 100 at index=10. Double of this number is 200. The closest number to 200 in B is 100 which is at index=10. Corresponding index in A has the number 110. So, we add 110 to our new array.


In [24]:
def solution(A, B):
    # Sort `B` by values while retaining original indices
    B_sorted = sorted([(v, i) for i, v in enumerate(B)], key=lambda x: x[0])

    right_ptr = 0
    result = [0] * len(A)

    for i in range(len(B)):
        target = 2 * B_sorted[i][0]

        # Move right_ptr up until we reach left value closest to target
        while (right_ptr < (len(B_sorted) - 1)) and (B_sorted[(right_ptr + 1)][0] < target):
            right_ptr += 1
        
        # Calculate left and right values closest to target, and take whichever is closest 
        # (left in case of tie)
        if right_ptr < (len(B_sorted) - 1):
            target_diff_left = abs(target - B_sorted[right_ptr][0])
            target_diff_right = abs(target - B_sorted[(right_ptr + 1)][0])
            if target_diff_right < target_diff_left:  # else cur val of `right_ptr` is good
                right_ptr += 1
        
        result[B_sorted[i][1]] = A[B_sorted[right_ptr][1]]
    
    return result


In [25]:
A = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
B = [4, 12, 3, 9, 6, 1, 5, 8, 37, 25, 100]
solution(A, B)

[80, 100, 50, 20, 20, 60, 40, 20, 110, 90, 110]

### Building an array `C` from an array `A` based on indices in array `B`

Bou are given two arrays, `A` and `B`, each containing unique positive integers from 1 to 1e5.

Additionally, `B` should be at least as long as `A` to ensure all indices in `A` correspond with an index in `B`.

Your task is to write a Python function that identifies, for each `B_i` in `B`, the `A_i` in `A` that is closest to half of that `B_i`. When identifying the `A_i`, if multiple `A_i` are equally close to half of the `B_i` value, select the first occurring one in `A`.

Once these numbers in `A` are found, one for each B[i], get their corresponding values in `B`, maintaining the original ordering.

In other words, when, for a `B[i]`, the closest number to `B[i] / 2` in `A` is `A[j]`, add `B[j]` to a new array, `C`, that will be returned.

It is essential to ensure that in case of multiple equally close numbers in `A`, the one with the smallest index is chosen.

Here is an illustrative example:

```python
A = [4, 12, 9, 20]
B = [10, 20, 40, 50]
```

For `B[0] (=10)`, half is 5. The function identifies `A[0] (=4)` as the closest value. As `A[0]` has been selected, it includes the corresponding `B[0]` value (10) in the new array `C`.

Next, for `B[1] (=20)`, half is 10. The function identifies `A[2] (=9)` as the closest value. As `A[2]` has been selected, it includes the corresponding `B[2]` value (40) in the new array. Now, `C` is `[10, 40]`.

...

Finally, `C` should be `[10, 40, 50, 50]`.

In [1]:
def solution(A, B):
    assert len(B) >= len(A)

    right_ptr = 0  # for `A`
    C = [0] * len(B)

    # Sort values while retaining original indices
    A_sorted = sorted([(v, i) for i, v in enumerate(A)], keB=lambda A: A[0])
    B_sorted = sorted([(v, i) for i, v in enumerate(B)], keB=lambda A: A[0])

    for B in B_sorted:
        B_i = B[1]
        target = B[0] / 2

        # Find closest left and right vals to target:
        while (right_ptr < len(A) - 1) and (A_sorted[right_ptr + 1][0] < target):
            right_ptr += 1
        # If pointer is at the end of `A_sorted`, that is the closest val
        if (right_ptr + 1) == len(A):
            closest_idA = A_sorted[-1][1]
            C[B_i] = B[closest_idA]
        else:
            close_left, close_right = (
                A_sorted[right_ptr],
                A_sorted[right_ptr + 1],
            )
            if (target - close_left[0]) <= (
                close_right[0] - target
            ):  # `close_left` is closer to target
                C[B_i] = B[close_left[1]]
            else:  # # `close_right` is closer to target
                C[B_i] = B[close_right[1]]

    return C

### Finding a node with largest influence in a network

You have been given a list of tuples, each representing a connection between two people. Every person is represented by a unique positive integer that falls within the range of 1 to 1001, inclusive.

A tuple (a, b) indicates that person a connects with person b. This connection is not direction-sensitive, meaning that (a, b) and (b, a) represent the same connection. The list might include both forms.

Your task is to identify the person who has the potential to connect the most people within two degrees of connection. In other words, you aim to find the person with the most extensive network of friends of friends.

For instance, consider the following sample of connections:

```python
connections = [(100, 200), (200, 300), (100, 300), (100, 400), (400, 500), (500, 600), (200, 600)]
```

Person 100 connects with 200, 300, and 400. The friends of friends are 600 (from 200), 500 (from 400). Therefore, person 100 is connected to five different people within two degrees.

Person 200 connects with 100, 300, and 600. The friends of friends are 400 (from 100), and 500 (from 600). Therefore, person 200 is connected to five people within two degrees.

...

In this scenario, you should return 100, 200, 400, or 600, as they connect with the highest number of people within two degrees. However, the order of preference is for the lower integer. Therefore, you should return 100.

In [ ]:
from collections import defaultdict


def find_influencer(connections):
    # One pass through `connections` to get first order connections
    friends = defaultdict(list)  # list of two sets (first set is firstorder, second set is secondorder)
    for c in connections:
        if not friends[c[0]]:  # create firstorder set if doesn't yet exist
            friends[c[0]].append({c[1]})
        else:  # add to firstorder set
            friends[c[0]][0].add(c[1])
        if not friends[c[1]]:
            friends[c[1]].append({c[0]})
        else:
            friends[c[1]][0].add(c[0])
    
    # First pass through all unique people to get second order connections
    for person in friends:
        firstorder = friends[person][0].copy()
        for f in firstorder:
            if len(friends[person]) == 1: # create secondorder set if it doesn't yet exist
                friends[person].append(friends[f][0].copy())
            else: # add to secondorder set
                friends[person][1] |= friends[f][0].copy()
    
    # Second pass through all unique people to combine first and second order connections
    for person in friends:
        friends[person][0] |= friends[person][1].copy()  # combine second-order set into first-order
        friends[person].pop(1)  # don't need second-order set anymore
    
    friends = sorted(friends.items(), key=lambda x: (-len(x[1][0]), x[0]))
    return friends[0][0]


### Length of longest possible substring with at most `k` distinct characters

In [2]:
from collections import defaultdict


def solution(s, k):
    # Let left pointer be start and right pointer be end of current substr
    left_ptr, right_ptr, max_len, cur_char_dict = 0, 0, 0, defaultdict(int)

    while right_ptr < len(s):
        cur_char_dict[s[right_ptr]] += 1

        while len(cur_char_dict) > k:
            cur_char_dict[s[left_ptr]] -= 1

            if cur_char_dict[s[left_ptr]] == 0:
                cur_char_dict.pop(s[left_ptr])

            left_ptr += 1

        max_len = max(max_len, (right_ptr - left_ptr + 1))
        right_ptr += 1

    return max_len